In [1]:
"""
找出RS在0.1，0.3，0.5，0.8，1时选出的算法名称
随机抽样算法在不同`{'dataset': 'spambase', 'metric': 'accuracy', 'dsr': 0.3} `
选出的前2个算法 ` {'models': ['adaboost', 'extra_trees']}`.
其他设置：

- 禁止降维（fsm=rf, fsr=1)
- 探索次数(30次)
 
"""
import os.path
import pandas as pd
from tshpo.lib_class import AnaHelper

# _file = AnaHelper.prepare_csv("c05_effect_of_sample_method_v2_original_20241025_0812.csv.gz", return_df=False)
_file = AnaHelper.prepare_csv("c05_effect_of_feature_method_v2_original_20250514_1607.csv.gz", return_df=False)
_generate_file = f"{_file}_acc.gz"
if not os.path.exists(_generate_file):
    _odf = pd.read_csv(_file)
    _odf['metric'] = "accuracy"
    _odf.to_csv(_generate_file)
df = pd.concat([
    AnaHelper.load_acc_select(os.path.basename(_file), alpha=0.005),
    AnaHelper.load_acc_select(os.path.basename(_generate_file), alpha=0.005)])
df

Parse configs


  0%|          | 0/480 [00:01<?, ?it/s]

KeyboardInterrupt



In [4]:
fdf = df[(df['dsm'] == "RS") & (df['top_n'] == 2)]
fdf

,top_n,acc,dataset,metric,fsm,fsr,dsm,dsr,n_exploration,hpo_opt_method,top_models_tshpo
1,2,1,pc4,roc_auc,ParamRepulsor,0.1,RS,1,30,RS,"[adaboost, extra_trees]"
9,2,1,pc4,roc_auc,ParamRepulsor,0.3,RS,1,30,RS,"[mlp, extra_trees]"
17,2,1,pc4,roc_auc,ParamRepulsor,0.5,RS,1,30,RS,"[mlp, extra_trees]"
25,2,1,pc4,roc_auc,ParamRepulsor,0.8,RS,1,30,RS,"[liblinear_svc, mlp]"
33,2,1,pc4,roc_auc,ParamRepulsor,1.0,RS,1,30,RS,"[mlp, extra_trees]"
...,...,...,...,...,...,...,...,...,...,...,...
3801,2,1,phoneme,accuracy,SP,0.1,RS,1,30,RS,"[mlp, random_forest]"
3809,2,1,phoneme,accuracy,SP,0.3,RS,1,30,RS,"[mlp, extra_trees]"
3817,2,1,phoneme,accuracy,SP,0.5,RS,1,30,RS,"[mlp, extra_trees]"
3825,2,1,phoneme,accuracy,SP,0.8,RS,1,30,RS,"[mlp, extra_trees]"


In [6]:
from pyutils.kvdb.kvdb_mysql import KVDBMySQL

db = KVDBMySQL(table_name="q11_tshpo_top2_on_dataset_fsm")
for k, v in fdf.iterrows():
    _key = {
        "dataset": v['dataset'],
        'metric': v['metric'],
        'fsm': v['fsm'],
        'fsr': v['fsr'],
    }
    _val = {"models": v['top_models_tshpo']}
    print(_key, _val)
    db.add(_key, _val)

{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'ParamRepulsor', 'fsr': 0.1} {'models': ['adaboost', 'extra_trees']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'ParamRepulsor', 'fsr': 0.3} {'models': ['mlp', 'extra_trees']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'ParamRepulsor', 'fsr': 0.5} {'models': ['mlp', 'extra_trees']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'ParamRepulsor', 'fsr': 0.8} {'models': ['liblinear_svc', 'mlp']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'ParamRepulsor', 'fsr': 1.0} {'models': ['mlp', 'extra_trees']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'UMAP', 'fsr': 0.1} {'models': ['extra_trees', 'random_forest']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'UMAP', 'fsr': 0.3} {'models': ['random_forest', 'extra_trees']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'UMAP', 'fsr': 0.5} {'models': ['extra_trees', 'random_forest']}
{'dataset': 'pc4', 'metric': 'roc_auc', 'fsm': 'UMAP', 'fsr': 0.8} {'models': ['extra_trees', 'random_